# **Tutorial** - Topic Modeling with BERTopic

In this tutorial we will be exploring how to use BERTopic to create topics from a dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

## What is Topic Modelling and Why is it Useful

In machine learning (ML) and natural language processing (NLP), topic modeling is an unsupervised statistical method for discovering abstract ‘topics’ that exist within a collection of documents.

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [2]:
%%capture
!pip install bertopic datasets

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use [_A Real-Life Scenario Dialogue Summarization Dataset_](https://github.com/cylnlp/dialogsum)


In [1]:
from datasets import load_dataset

docs = load_dataset('knkarthick/dialogsum')['train']['summary']

ModuleNotFoundError: ignored

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [5]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/390 [00:00<?, ?it/s]

2022-09-01 09:28:04,895 - BERTopic - Transformed documents to Embeddings
2022-09-01 09:28:41,185 - BERTopic - Reduced dimensionality
2022-09-01 09:28:58,373 - BERTopic - Clustered reduced embeddings


## What Happened in the Previous Step
- Transformed documents to Embeddings
- Reduced dimensionality with [UMAP](https://umap-learn.readthedocs.io/en/latest/how_umap_works.html), which is relatively fast even on larger datasets with high (3+) dimensionality and similar points tend to cluster together in reduced dimensionality ([Video](https://youtu.be/eN0wFzBA4Sc))
- Clustered reduced embeddings using [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html)

**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [38]:
freq = topic_model.get_topic_info(); freq.head(20)

,Topic,Count,Name
0,-1,4263,-1_to_the_and_person1
1,0,511,0_mary_her_she_invites
2,1,471,1_job_company_interview_position
3,2,384,2_food_orders_order_restaurant
4,3,257,3_apartment_house_rent_move
5,4,251,4_school_class_university_students
6,5,234,5_medicine_doctor_sleep_prescription
7,6,182,6_game_play_football_basketball
8,7,180,7_movie_movies_film_watch
9,8,166,8_hotel_room_double_reservation


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [39]:
topic_model.get_topic(13)  # Select the most frequent topic

[('concert', 0.049852515972623045),
 ('music', 0.048690460365645505),
 ('band', 0.024562466838165518),
 ('singer', 0.021311108720176174),
 ('song', 0.01804757170203801),
 ('classical', 0.017048886976140938),
 ('sing', 0.016824196248757858),
 ('piano', 0.014037000212696228),
 ('guitar', 0.012309276815500023),
 ('play', 0.012257726532156326)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [40]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [44]:
topic_model.visualize_distribution(probs[10], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [45]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [46]:
topic_model.visualize_barchart(top_n_topics=10)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [47]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [48]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [49]:
topic_model.update_topics(docs, topics, n_gram_range=(1, 2))

In [51]:
topic_model.get_topic(1)   # We select topic that we viewed before

[('job', 0.016989637463644115),
 ('company', 0.011353295117043428),
 ('the job', 0.009621241819271546),
 ('interview', 0.00889811310290724),
 ('work', 0.007467856866595953),
 ('person2', 0.0074636697480689575),
 ('position', 0.007151494830360274),
 ('boss', 0.006809162716150451),
 ('the company', 0.006412551329190203),
 ('working', 0.005857230949284296)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [17]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probs, nr_topics=60)

2022-08-31 14:02:45,257 - BERTopic - Reduced number of topics from 180 to 61


# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [93]:
similar_topics, similarity = topic_model.find_topics("smell of onions", top_n=5); similar_topics

[29, 133, 77, 52, 81]

In [94]:
topic_model.get_topic(29)

[('smoking', 0.07511039309866917),
 ('quit smoking', 0.028475883385382757),
 ('quit', 0.025879479636898614),
 ('smoke', 0.020024965891088452),
 ('smoking person1', 0.017551001973455316),
 ('cigarette', 0.017551001973455316),
 ('non smoking', 0.016017684404277803),
 ('non', 0.014758615047429683),
 ('smoking is', 0.014747260007635266),
 ('to quit', 0.013683630965035094)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
